In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from utils import (
    create_fasta_file, create_random_strand, reverse_complement,
    read_synthesized_strands_from_file, get_fastq_records, get_badread_strand_id,
)
from checksum import CheckSum4
import random
from clustering import Clustering
import numpy as np
from strand_reconstruction import align
import sys
from utils import reverse_complement
np.set_printoptions(threshold=sys.maxsize)
import os

In [3]:
# Read cat.fasta - add encoding
# write fasta
# badread
# do clustering get candidates
# decode without apriori

RU10_path = "/Users/i/Downloads/DNA_STorage/code/libjpeg/cat_FFDX.jpg_RU10.fasta"
checksum_path = os.path.join(os.path.dirname(RU10_path), "cat_FFDX_checksum.fasta")
recovered_path = os.path.join(os.path.dirname(RU10_path), "cat_FFDX_rec.fasta")
oligoLen = 204

checksum = CheckSum4(reference_length=oligoLen)
original_strands, original_strand_ids = read_synthesized_strands_from_file(file_path=RU10_path)
encoded_strands = checksum.encode(original_strands)
create_fasta_file(ids=original_strand_ids, strands=encoded_strands, output_filepath=checksum_path)
print("Checksum encoded strands saved to", checksum_path)

Checksum encoded strands saved to /Users/i/Downloads/DNA_STorage/code/libjpeg/cat_FFDX_checksum.fasta


### BADREAD & unzip

In [135]:
# Raptor
records = get_fastq_records(fastq_filepath="/Users/i/Downloads/DNA_STorage/code/libjpeg/reads.fastq")

reads = [str(i.seq) for i in records]
ids = [get_badread_strand_id(i) for i in records]

clustering = Clustering(strand_pool=reads, reference_length=oligoLen+4, original_strands=encoded_strands, strand_pool_ids=ids, distance_threshold=40)
clustering.run_pipeline(eval=True)
# We want to recover the original strands after checksum
decoded_strands = checksum.decode(candidates=clustering.candidates, n_reference_strands=len(original_strands), clustered_seqs=clustering.clustered_seqs, n_guesses=5, guesses=True)
ids = ["" for i in range(len(decoded_strands))]
create_fasta_file(ids, decoded_strands, output_filepath=recovered_path)

18521it [00:00, 48750.99it/s] 


Filtering strands by length
0.042438313266022355 strands filtered out
Removing adapters


100%|██████████| 17735/17735 [00:00<00:00, 38058.85it/s]


Clustering strands
Total strands 17735


100%|██████████| 17735/17735 [00:43<00:00, 406.03it/s]


...

histt


100%|██████████| 959/959 [00:00<00:00, 7025.27it/s]


484 direct checksum matches found
Making guesses


100%|██████████| 475/475 [01:18<00:00,  6.09it/s]

229 extra found after guessing
0.48807854137447404 were reversed
713 Valid checksum strands found


In [136]:
# BEFORE CHECKSUM
import plotly.figure_factory as ff

df_50 = clustering.evaluate_candidates(None, hist=False)
fig = ff.create_distplot([df_50["reference_recoveries"]], [""], colors=['#835AF1'], bin_size=0.05, show_rug=False, show_hist=True, curve_type="kde")
fig.show()

959it [00:04, 196.02it/s]


In [134]:
# BEFORE CHECKSUM
import plotly.figure_factory as ff

df_100 = clustering.evaluate_candidates(None, hist=False)
fig = ff.create_distplot([df_100["reference_recoveries"]], [""], colors=['#835AF1'], bin_size=0.05, show_rug=False, show_hist=True, curve_type="kde")
fig.show()

959it [00:04, 194.71it/s]


In [104]:
# We want to recover the original strands after checksum
decoded_strands = checksum.decode(candidates=clustering.candidates, n_reference_strands=len(original_strands), clustered_seqs=clustering.clustered_seqs, n_guesses=5, guesses=True)
ids = ["" for i in range(len(decoded_strands))]
create_fasta_file(ids, decoded_strands, output_filepath=recovered_path)

100%|██████████| 959/959 [00:00<00:00, 32684.92it/s]


442 direct checksum matches found
Making guesses


100%|██████████| 517/517 [01:45<00:00,  4.91it/s]

91 extra found after guessing
0.4652908067542214 were reversed
533 Valid checksum strands found


In [ ]:
df = clustering.evaluate_candidates(None, hist=True)

959it [00:06, 145.23it/s]

histt


In [108]:
clustering.original_strands = original_strands
clustering.fsm(candidates=decoded_strands)
df = clustering.evaluate_candidates(decoded_strands, hist=True)

Found 528


533it [00:00, 5672.51it/s]

histt


In [87]:
import plotly.figure_factory as ff

print(df.keys())
print(df["reference_strand_indices"])
print(df["recovery_rates"])
help = np.delete(df["reference_recoveries"], df["reference_strand_indices"])
d32 = [help, df["recovery_rates"]]

dict_keys(['reference_recoveries', 'reference_strand_indices', 'recovery_rates'])
[889 358 372 516 797  93 596 293 333  46 855 811 618 761 448 388 180 626
 686 850 555 574 331 645 806 310 751 586 362 183 732 440 409 528 518  39
 143 706 346 363 522 627 184 157 568 708 292 572 721 742 661 696 622 699
  83 825 810 834 205 765 766 425 868   8 607  38 224 102 168 540 343 339
 804 231  15 311 796 519 365 243 494   7 791 648 378 186  30 819 799 612
 609 878 786 351 412  87  80 153 366 179 120 736 220 155 502 511 559 705
 552 262 245 235 406 442 480 549 300 784 506 221 813 394 562 716  85 817
 307  99 482 129 402  96 261 100 758 219 866 357 113 396 678  20 698 668
 499 338 840 779 818 109 457 795 523 872 281 871 883 419 814  66 330 368
 467 783 517 283 842 693 370 233 665 592 722 662 405 598 670 887 860 739
 192 176 429  25 752 737 725 456  55  73  18 541 514 515 488 507 673 803
 588 355 600   0 258 246 681  29 214 265 308 774 136 865 337 577 801 836
  84 175 305 899 142 278 536 430 483 908  

In [132]:
fig = ff.create_distplot([d2[1], d32[1]], ['d2','d32'], colors=['#333F44', '#37AA9C'], bin_size=0.05, show_rug=True, curve_type="normal")
fig.show()

In [ ]:
d2
d4
d8
d16
d32

In [88]:
fig = ff.create_distplot(d32, ['erased','recovered'], colors=['#333F44', '#37AA9C'], bin_size=0.05, show_rug=True, curve_type="normal")
fig.show()

/Users/i/miniconda3/envs/NOREC/lib/python3.10/site-packages/scipy/stats/_distn_infrastructure.py:2093: RuntimeWarning:

invalid value encountered in divide



In [58]:
import plotly.figure_factory as ff
fig = ff.create_distplot(d2, ['erased','recovered'], colors=['#333F44', '#37AA9C'], bin_size=0.05, show_rug=True)
fig.update_layout(
    title_text=f"Recovery of reference strands for candidates after checksum",
    plot_bgcolor='white',
    )
# fig.update_layout(
#     margin=dict(l=20, r=20, t=20, b=20),
#     xaxis=dict(title=dict(text=r"Number of oligos received")),
#     yaxis=dict(title=dict(text=r"Chunk Index")),
#     plot_bgcolor='white',
#     font_family="Arial",
#     font_color="black",
#     font_size=16,)
fig.update_xaxes(
# range=[0, 1],
mirror=True,
ticks='outside',
showline=True,
linecolor='black',
gridcolor='lightgrey'
)
fig.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey',
)
fig.show()

LinAlgError: The data appears to lie in a lower-dimensional subspace of the space in which it is expressed. This has resulted in a singular data covariance matrix, which cannot be treated using the algorithms implemented in `gaussian_kde`. Consider performing principle component analysis / dimensionality reduction and using `gaussian_kde` with the transformed data.

In [17]:
import plotly.graph_objs as go
import pandas as pd
import plotly.express as px
import plotly.io as pio

pio.write_image(fig, "/Users/i/Desktop/recovery.png", scale=3, width=1080, height=720)